# Topic and word list from a working paper by Sophie Stone of the Dartmouth College Economics Department



## https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [ ]:
# Dependencies
# !pip install "calcbench-api-client[Pandas, Backoff, BeautifulSoup]" sentence-transformers tqdm qgrid ipywidgets

In [21]:
import warnings

import pandas as pd
import itertools
from tqdm.notebook import tqdm
import qgrid

import torch

# https://github.com/UKPLab/sentence-transformers
from sentence_transformers import SentenceTransformer, util
import calcbench as cb

model = SentenceTransformer("distilbert-base-nli-stsb-mean-tokens")

cb.enable_backoff()
tqdm.pandas(desc="progress")

In [2]:
categories = [
    (
        "environmental",
        """biodiversity, carbon, carbon negative, carbon neutral, carbon zero, clean-up, clean, clean tech, climate change, climate positive, contamination, discharge, emission, energy-efficient, environmental, environmental risk, fuel, fuel efficiency, green, greenhouse gas, hazardous, low carbon, natural resource, net-zero emission, pollution, remediation, sustainability, sustainable, toxic, waste, water, zero carbon, zero net carbon""",
    ),
    (
        "social",
        """accident, antiracism, consumer protection, customer privacy, employee relation, equal, equal pay, equity, gender equality, health, human right, justice, labor relation, labor standard, racial equity, racial awareness, racial justice, working condition""",
    ),
    (
        "governance",
        """advocacy, antitrust, board independence, code of ethic, compensation, corporate culture, corporate governance, corruption, governance risk, justice, political lobbying, scandal, shareholder right, stability, stewardship, transparency""",
    ),
]

In [3]:
with tqdm() as progress_bar:
    disclosures = cb.document_dataframe(
        company_identifiers=["msft", "orcl"],  # "DJIA"
        disclosure_names=["ManagementsDiscussionAndAnalysis", "RiskFactors"],
        all_history=True,
        period_type="annual",
        progress_bar=progress_bar,
        # entire_universe=True,
    )

0it [00:00, ?it/s]

In [4]:
def get_contents(d):
    try:
        return d.get_contents_text()
    except Exception as e:
        print(f"Exception getting {d} \n{e}")


disclosure_contents = disclosures.progress_applymap(
    get_contents, na_action="ignore"
).fillna("")

progress:   0%|          | 0/44 [00:00<?, ?it/s]

In [9]:
disclosure_embeddings = disclosure_contents.progress_applymap(
    lambda c: model.encode(c, convert_to_tensor=True)
)

progress:   0%|          | 0/44 [00:00<?, ?it/s]

In [53]:
def distance(column_embeddings: pd.Series, category_embeddings):
    distances = util.pytorch_cos_sim(torch.stack(list(column_embeddings)), category_embeddings).T[0]
    return distances

In [54]:
distances = []
for _, category_words in tqdm(categories):
    category_embeddings = model.encode(category_words, convert_to_tensor=True)
    category_distances = disclosure_tensors.progress_apply(distance, category_embeddings=category_embeddings)
    distances.append(category_distances)

  0%|          | 0/3 [00:00<?, ?it/s]

progress:   0%|          | 0/4 [00:00<?, ?it/s]

progress:   0%|          | 0/4 [00:00<?, ?it/s]

progress:   0%|          | 0/4 [00:00<?, ?it/s]

In [51]:
pd.concat(distances, axis=1, keys=[category for category, _ in categories])

environmental                        \
disclosure_type_name ManagementsDiscussionAndAnalysis           RiskFactors   
ticker                                           MSFT      ORCL        MSFT   
period                                                                        
2010                                         0.067704  0.046842    0.041494   
2011                                         0.056875  0.021732    0.031076   
2012                                         0.036274 -0.012102    0.125279   
2013                                         0.056632  0.041432    0.126557   
2014                                         0.096285  0.029410    0.172148   
2015                                         0.097422  0.058823    0.138491   
2016                                         0.108320  0.083348    0.125647   
2017                                         0.135766  0.035972    0.138491   
2018                                         0.135766  0.040085    0.138491   
2019                                         0.109887  0.063685    0.138491   
2020                                         0.109887  0.063685    0.138491   

                                                         social            \
disclosure_type_name           ManagementsDiscussionAndAnalysis             
ticker                    ORCL                             MSFT      ORCL   
period                                                                      
2010                  0.069147                         0.287402  0.282850   
2011                  0.069147                         0.317599  0.289613   
2012                  0.076744                         0.287051  0.285193   
2013                  0.085984                         0.286282  0.321872   
2014                  0.076744                         0.026628  0.321093   
2015                  0.076744                         0.283516  0.322281   
2016                  0.122130                         0.285077  0.354271   
2017                  0.115558                         0.288576  0.322182   
2018                  0.122130                         0.288576  0.325882   
2019                  0.057861                         0.280293  0.351092   
2020                  0.050335                         0.280293  0.351092   

                                                                 governance  \
disclosure_type_name RiskFactors           ManagementsDiscussionAndAnalysis   
ticker                      MSFT      ORCL                             MSFT   
period                                                                        
2010                    0.390782  0.374338                         0.434765   
2011                    0.388511  0.374338                         0.406364   
2012                    0.397964  0.372680                         0.386091   
2013                    0.405035  0.385839                         0.390310   
2014                    0.400975  0.372680                         0.035563   
2015                    0.408795  0.372680                         0.416288   
2016                    0.397462  0.372968                         0.401584   
2017                    0.408795  0.365560                         0.428353   
2018                    0.408795  0.372968                         0.428353   
2019                    0.408795  0.368606                         0.415492   
2020                    0.408795  0.366652                         0.415492   

                                                      
disclosure_type_name           RiskFactors            
ticker                    ORCL        MSFT      ORCL  
period                                                
2010                  0.399483    0.302868  0.367925  
2011                  0.403239    0.310657  0.367925  
2012                  0.401582    0.376065  0.382831  
2013                  0.405992    0.363333  0.375781  
2014                  0.412648    0.396679  0.382831  
2015             